For working with the model, we first import some packages and the modelclass:

In [ ]:
#os.chdir('C:\\Users\\mpher\\Documents\\Uni\\Master\\02_Exchange\\01_Academics\\Energy Economics of the Green Transition\\0_Final Project\EnergyEconGroupWork')

In [ ]:
import os
current_directory = os.getcwd()
print(current_directory)

In [1]:
%run ./stdPackages.ipynb
from pyDbs import *

os.chdir(os.path.join(d['curr'], 'py'))

from mBasicPH_storage import *
os.chdir(d['curr'])

Next we load the database:

In [3]:
kwargs = {'variables': ['Fundamentals', 'LoadVariables', 'GeneratorsVariables','StorageVariables'],
        'maps': ['LoadMaps','GeneratorsMaps','StorageMaps'],
        'variable2D': ['HourlyVariation'],
        'scalars': ['Scalars']}

path_to_data = os.path.join(d['curr'], 'Data', 'mBasicPH_storage_Data.xlsx')

db = read.dbFromWB(path_to_data, kwargs)
readSets(db)

Convert variables to floats:

In [4]:
[db.__setitem__(var,db[var].astype(float).sort_index()) for var in db.getTypes('variable').keys()];

We need to read in heat areas manually:

In [5]:
db['g_H'] = db['id2g_H'].get_level_values('g_H')

In [6]:
db['g_H']

Index(['DK_Central', 'DK_Central', 'DK_Central', 'DK_Central', 'DK_Central',
       'DK_Central', 'DK_Central', 'DK_Central', 'DK_Central', 'DK_Central',
       'DK_Central', 'DK_Central', 'DK_Central', 'DK_Central', 'DK_Central',
       'DK_Central', 'DK_Central', 'DK_Central', 'DK_Central', 'DK_Central',
       'DK_Decentral', 'DK_Decentral', 'DK_Decentral', 'DK_Decentral',
       'DK_Decentral', 'DK_Decentral', 'DK_Decentral', 'DK_Decentral',
       'DK_Decentral', 'DK_Decentral', 'DK_Decentral', 'DK_Decentral',
       'DK_Decentral', 'DK_Decentral', 'DK_Decentral', 'DK_Decentral',
       'DK_Decentral', 'DK_Decentral', 'DK_Decentral', 'DK_Decentral',
       'DK_Decentral'],
      dtype='object', name='g_H')

We need to read in electricity areas manually:

In [10]:
db['g_E'] = db['id2g_E'].get_level_values('g_E')

In [11]:
db['g_E']

Index(['DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK',
       'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK',
       'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK'],
      dtype='object', name='g_E')

Collect all areas in one:

In [12]:
db['g'] = db['g_E'].copy()

In [13]:
db['g']

Index(['DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK',
       'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK',
       'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK'],
      dtype='object', name='g_E')

Finally, we will assume that linelosses and trade are zero such that the marginal willingness to pay and marginal generation costs of "transmission lines" does not have to be corrected. 

We then initialize and solve the model:

In [14]:
m = mSimple(db)
m.db.symbols.keys()

dict_keys(['EmissionTax', 'EmissionIntensity', 'FuelPrice', 'LoadVariation_E', 'Load_H', 'LoadVariation_H', 'Load_E', 'GeneratingCap_E', 'FuelMix', 'GeneratingCap_H', 'FOM', 'OtherMC', 'E2H', 'selfDischarge', 'chargeCap_H', 'sCap', 'effC', 'effD', 'c_H2g', 'c_E2g', 'id2tech', 'id2g_E', 'id2hvt', 'id2g_H', 'tech2modelTech', 'id2g', 'CapVariation', 'MWP_E', 'MWP_H', 'EmissionType', 'BFt', 'c_E', 'h', 'c_H', 'id', 'hvt', 'g_H', 'g', 'g_E', 'id2modelTech2tech'])

In [ ]:
variable_names = ['FuelPrice', 'EmissionIntensity', 'EmissionTax', 'LoadVariation_H', 'MWP_H', 'Load_E', 'MWP_E', 
                  'LoadVariation_E', 'Load_H', 'effD', 'sCap', 'InvestCost', 'selfDischarge', 'effC', 'OtherMC', 
                  'GeneratingCap_H', 'FOM', 'chargeCap_H', 'GeneratingCap_E', 'E2H', 'FuelMix', 'c_H2g_H', 'c_E2g_E', 
                  'id2tech', 'tech2modelTech', 'id2g_E', 'id2g_H', 'id2hvt', 'g_E2g_H', 'g_E2g', 'g_H2g', 'h2Month', 
                  'h2Weekday', 'h2Week', 'h2HourOfTheDay', 'CapVariation', 'BFt', 'EmissionType', 'c_H', 'h', 'c_E', 
                  'id', 'tech', 'hvt', 'g_H', 'g_E', 'g', 'id2modelTech2tech', 'mc']

In [15]:
m.db['g_E']

Index(['DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK',
       'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK',
       'DK', 'DK', 'DK', 'DK', 'DK', 'DK', 'DK'],
      dtype='object', name='g_E')

In [ ]:
#m()

In [ ]:
#df = pd.DataFrame(db['x'])
#df.to_excel(os.path.join(d['curr'], 'x.xlsx'), engine='openpyxl')

#db['x']

What to plot?

- CO2 Emissions
- Usage of Storage
- Marginal Economic value of storage
- seasonal storage?